In [1]:
#part of the code is from https://github.com/maciejkula/triplet_recommendations_keras
#based on code provided on Kaggle kernel
import numpy as np
import pandas as pd
import os
import sys
import glob
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from PIL import Image
from keras import backend as K
from keras import optimizers, losses, activations, models
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Embedding, Flatten, Input, merge
from keras.layers import Conv2D, MaxPooling2D, Dense, GlobalMaxPooling2D
from keras.layers import Convolution2D, Dropout, BatchNormalization, \
                            GlobalMaxPool2D, Concatenate, GlobalAveragePooling2D, Lambda
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras.applications.resnet50 import ResNet50
from keras.preprocessing.image import random_rotation, random_shift, random_shear, random_zoom, \
                            random_channel_shift, transform_matrix_offset_center, img_to_array

Using TensorFlow backend.


In [2]:
batch_size = 8
input_shape = (256, 256)
base_path = "../dataset/train/"
file_path = "triplet_model_with_data_aug_weights.best.hdf5"

In [3]:
class sample_gen(object):
    def __init__(self, file_class_mapping, other_class = "new_whale"):
        self.file_class_mapping= file_class_mapping
        self.class_to_list_files = defaultdict(list)
        self.list_other_class = []
        self.list_all_files = list(file_class_mapping.keys())
        self.range_all_files = list(range(len(self.list_all_files)))

        for file, class_ in file_class_mapping.items():
            #if class_ == other_class:
            #    self.list_other_class.append(file)
            #else:
            self.class_to_list_files[class_].append(file)

        self.list_classes = list(set(self.file_class_mapping.values()))
        self.range_list_classes= range(len(self.list_classes))
        
        #FINAL add new_whale
        self.class_weight = np.array([len(self.class_to_list_files[class_]) for class_ in self.list_classes])
        self.class_weight = self.class_weight/np.sum(self.class_weight)

    def get_sample(self):
        class_idx = np.random.choice(self.range_list_classes, 1, p=self.class_weight)[0]
        #FINAL not new whale, choose any two whales in class (could be same)
        if self.list_classes[class_idx] != "new_whale":
            examples_class_idx = np.random.choice(range(len(self.class_to_list_files[self.list_classes[class_idx]])), 2)
            positive_example_1, positive_example_2 = \
                self.class_to_list_files[self.list_classes[class_idx]][examples_class_idx[0]],\
                self.class_to_list_files[self.list_classes[class_idx]][examples_class_idx[1]]
        #FINAL new whale, use same sample
        else:
            examples_class_idx = np.random.choice(range(len(self.class_to_list_files["new_whale"])), 1)
            positive_example_1, positive_example_2 = \
                self.class_to_list_files["new_whale"][examples_class_idx[0]],\
                self.class_to_list_files["new_whale"][examples_class_idx[0]]

        negative_example = None
        while negative_example is None or \
                (self.file_class_mapping[negative_example] == self.file_class_mapping[positive_example_1] and self.file_class_mapping[positive_example_1] != "new_whale") or \
                (negative_example == positive_example_1):
            negative_example_idx = np.random.choice(self.range_all_files, 1)[0]
            negative_example = self.list_all_files[negative_example_idx]
        return positive_example_1, negative_example, positive_example_2

In [4]:
def identity_loss(y_true, y_pred):

    return K.mean(y_pred - 0 * y_true)

In [5]:
def bpr_triplet_loss(X):
    positive_item_latent, negative_item_latent, user_latent = X
    # BPR loss
    loss = 1.0 - K.sigmoid(
        K.sum(user_latent * positive_item_latent, axis=-1, keepdims=True) -
        K.sum(user_latent * negative_item_latent, axis=-1, keepdims=True))

    return loss

In [6]:
def get_base_model():
    latent_dim = 50
    base_model = ResNet50(include_top=False, weights='imagenet') # use weights='imagenet' locally

    # for layer in base_model.layers:
    #     layer.trainable = False

    x = base_model.output
    x = GlobalMaxPooling2D()(x)
    x = Dropout(0.5)(x)
    dense_1 = Dense(latent_dim)(x)
    normalized = Lambda(lambda  x: K.l2_normalize(x, axis=1))(dense_1)
    base_model = Model(base_model.input, normalized, name="base_model")
    return base_model

In [7]:
def build_model():
    base_model = get_base_model()

    positive_example_1 = Input(input_shape+(3,) , name='positive_example_1')
    negative_example = Input(input_shape+(3,), name='negative_example')
    positive_example_2 = Input(input_shape+(3,), name='positive_example_2')

    positive_example_1_out = base_model(positive_example_1)
    negative_example_out = base_model(negative_example)
    positive_example_2_out = base_model(positive_example_2)

    loss = merge(
        [positive_example_1_out, negative_example_out, positive_example_2_out],
        mode=bpr_triplet_loss,
        name='loss',
        output_shape=(1, ))

    model = Model(
        input=[positive_example_1, negative_example, positive_example_2],
        output=loss)
    model.compile(loss=identity_loss, optimizer=Adam(0.000001))

    print(model.summary())

    return model

In [8]:
def build_inference_model(weight_path=file_path):
    base_model = get_base_model()

    positive_example_1 = Input(input_shape+(3,) , name='positive_example_1')
    negative_example = Input(input_shape+(3,), name='negative_example')
    positive_example_2 = Input(input_shape+(3,), name='positive_example_2')

    positive_example_1_out = base_model(positive_example_1)
    negative_example_out = base_model(negative_example)
    positive_example_2_out = base_model(positive_example_2)

    loss = merge(
        [positive_example_1_out, negative_example_out, positive_example_2_out],
        mode=bpr_triplet_loss,
        name='loss',
        output_shape=(1, ))

    model = Model(
        input=[positive_example_1, negative_example, positive_example_2],
        output=loss)
    model.compile(loss=identity_loss, optimizer=Adam(0.000001))

    model.load_weights(weight_path)

    inference_model = Model(base_model.get_input_at(0), output=base_model.get_output_at(0))
    inference_model.compile(loss="mse", optimizer=Adam(0.000001))
    print(inference_model.summary())

    return inference_model

In [9]:
def read_and_resize(filepath):
    im = Image.open((filepath)).convert('RGB')
    im = im.resize(input_shape)
    im_array = np.array(im, dtype="uint8")[..., ::-1]
    return np.array(im_array / (np.max(im_array) + 0.001), dtype="float32")

In [10]:
def random_greyscale(img, p):
    if np.random.uniform(0, 1) < p:
        temp = np.dot(img[...,:3], [0.299, 0.587, 0.114])
        temp = np.stack((temp,) * 3, -1)
        return temp
    return img

In [11]:
def augment(im_array):
    #flip image
    if np.random.uniform(0, 1) > 0.5:
        im_array = np.fliplr(im_array)
        
    #FINAL add noise
    im_array = random_rotation(im_array, rg=30, row_axis=0, col_axis=1, channel_axis=2, fill_mode='nearest')
    #im_array = random_shift(im_array, wrg=0.1, hrg=0.3, row_axis=0, col_axis=1, channel_axis=2, fill_mode='nearest')
    im_array = random_shear(im_array, intensity=10, row_axis=0, col_axis=1, channel_axis=2, fill_mode='nearest')
    im_array = random_zoom(im_array, zoom_range=(1, 2.0), row_axis=0, col_axis=1, channel_axis=2, fill_mode='nearest')
    im_array = random_greyscale(im_array, 0.4)    
    
    return im_array

In [12]:
def gen(triplet_gen):
    while True:
        list_positive_examples_1 = []
        list_negative_examples = []
        list_positive_examples_2 = []

        for i in range(batch_size):
            positive_example_1, negative_example, positive_example_2 = triplet_gen.get_sample()
            positive_example_1_img, negative_example_img, positive_example_2_img = read_and_resize(base_path+positive_example_1), \
                                                                       read_and_resize(base_path+negative_example), \
                                                                       read_and_resize(base_path+positive_example_2)

            positive_example_1_img, negative_example_img, positive_example_2_img = augment(positive_example_1_img), \
                                                                                   augment(negative_example_img), \
                                                                                   augment(positive_example_2_img)

            list_positive_examples_1.append(positive_example_1_img)
            list_negative_examples.append(negative_example_img)
            list_positive_examples_2.append(positive_example_2_img)

        list_positive_examples_1 = np.array(list_positive_examples_1)
        list_negative_examples = np.array(list_negative_examples)
        list_positive_examples_2 = np.array(list_positive_examples_2)
        yield [list_positive_examples_1, list_negative_examples, list_positive_examples_2], np.ones(batch_size)

In [13]:
def data_generator(fpaths, batch=16):
    i = 0
    for path in fpaths:
        if i == 0:
            imgs = []
            fnames = []
        i += 1
        img = read_and_resize(path)
        imgs.append(img)
        fnames.append(os.path.basename(path))
        if i == batch:
            i = 0
            imgs = np.array(imgs)
            yield fnames, imgs
    if i < batch:
        imgs = np.array(imgs)
        yield fnames, imgs
    raise StopIteration()

In [14]:
data = pd.read_csv('../dataset/train.csv')
train, test = train_test_split(data, test_size=0.3, shuffle=True, random_state=1337)
file_id_mapping_train = {k: v for k, v in zip(train.Image.values, train.Id.values)}
file_id_mapping_test = {k: v for k, v in zip(test.Image.values, test.Id.values)}
train_gen = sample_gen(file_id_mapping_train)
test_gen = sample_gen(file_id_mapping_test)

# Prepare the test triplets
model = build_model()
#model.load_weights(file_path)

callbacks_list = []
callbacks_list.append(ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min'))
callbacks_list.append(EarlyStopping(monitor="val_loss", mode="min", patience=2))
epochs = 300
history = model.fit_generator(gen(train_gen),
                              validation_data=gen(test_gen),
                              epochs=epochs,
                              verbose=2,
                              workers=1,
                              callbacks=callbacks_list,
                              steps_per_epoch=300,
                              validation_steps=30)
                              
print("Done fitting\n", file=sys.stderr, flush=True)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
dim is deprecated, use axis instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:16: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  app.launch_new_instance()
/home/user/.local/lib/python3.5/site-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("lo...)`


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
positive_example_1 (InputLayer)  (None, 256, 256, 3)   0                                            
____________________________________________________________________________________________________
negative_example (InputLayer)    (None, 256, 256, 3)   0                                            
____________________________________________________________________________________________________
positive_example_2 (InputLayer)  (None, 256, 256, 3)   0                                            
____________________________________________________________________________________________________
base_model (Model)               (None, 50)            23690162    positive_example_1[0][0]         
                                                                   negative_example[0][0]  

Done fitting



Epoch 00025: val_loss did not improve
266s - loss: 0.3945 - val_loss: 0.3461


In [15]:


data = pd.read_csv('train.csv')

file_id_mapping = {k: v for k, v in zip(data.Image.values, data.Id.values)}

inference_model = build_inference_model()

train_files = glob.glob("../dataset/train/*.jpg")
test_files = glob.glob("../dataset/test/*.jpg")

#getting train data embedding
train_preds = []
train_file_names = []
i = 1
for fnames, imgs in data_generator(train_files, batch=32):
    print(i*32/len(train_files)*100)
    i += 1
    predicts = inference_model.predict(imgs)
    predicts = predicts.tolist()
    train_preds += predicts
    train_file_names += fnames
train_preds = np.array(train_preds)

#getting test data embedding
test_preds = []
test_file_names = []
i = 1
for fnames, imgs in data_generator(test_files, batch=32):
    print(i * 32 / len(test_files) * 100)
    i += 1
    predicts = inference_model.predict(imgs)
    predicts = predicts.tolist()
    test_preds += predicts
    test_file_names += fnames
test_preds = np.array(test_preds)

neigh = NearestNeighbors(n_neighbors=6)
neigh.fit(train_preds)
#distances, neighbors = neigh.kneighbors(train_preds)
#print(distances, neighbors)

distances_test, neighbors_test = neigh.kneighbors(test_preds)

distances_test, neighbors_test = distances_test.tolist(), neighbors_test.tolist()

preds_str = []

for filepath, distance, neighbour_ in zip(test_file_names, distances_test, neighbors_test):
    sample_result = []
    sample_classes = []
    for d, n in zip(distance, neighbour_):
        train_file = train_files[n].split(os.sep)[-1]
        class_train = file_id_mapping[train_file]
        sample_classes.append(class_train)
        sample_result.append((class_train, d))

    if "new_whale" not in sample_classes:
        sample_result.append(("new_whale", 0.1))
    sample_result.sort(key=lambda x: x[1])
    sample_result = sample_result[:5]
    preds_str.append(" ".join([x[0] for x in sample_result]))

df = pd.DataFrame(preds_str, columns=["Id"])
df['Image'] = [x.split(os.sep)[-1] for x in test_file_names]
df.to_csv(str(epochs) + "_epoch_with_data_aug.csv", index=False)



/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:16: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  app.launch_new_instance()
/home/user/.local/lib/python3.5/site-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("lo...)`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:25: UserWarning: Update your `Model` call to the Keras 2 API: `Model(Tensor("in..., outputs=Tensor("la...)`


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, None, None, 64 9472        input_2[0][0]                    
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, None, None, 64 256         conv1[0][0]                      
____________________________________________________________________________________________________
activation_50 (Activation)       (None, None, None, 64 0           bn_conv1[0][0]                   
___________________________________________________________________________________________

0.3248730964467005
0.649746192893401
0.9746192893401016
1.299492385786802
1.6243654822335025
1.9492385786802031
2.2741116751269037
2.598984771573604
2.9238578680203045
3.248730964467005
3.5736040609137056
3.8984771573604062
4.223350253807107
4.548223350253807
4.873096446700507
5.197969543147208
5.522842639593908
5.847715736040609
6.1725888324873095
6.49746192893401
6.82233502538071
7.147208121827411
7.472081218274111
7.7969543147208125
8.121827411167512
8.446700507614214
8.771573604060915
9.096446700507615
9.421319796954315
9.746192893401014
10.071065989847716
10.395939086294415
10.720812182741117
11.045685279187817
11.370558375634518
11.695431472081218
12.02030456852792
12.345177664974619
12.67005076142132
12.99492385786802
13.319796954314722
13.64467005076142
13.969543147208121
14.294416243654823
14.619289340101524
14.944162436548222
15.269035532994923
15.593908629441625
15.918781725888326
16.243654822335024
16.568527918781726
16.893401015228427
17.21827411167513
17.54314720812183
17

28.904548366431776
29.10954516335682
29.314541960281872
29.51953875720692
29.724535554131965
29.929532351057013
30.134529147982065
30.33952594490711
30.544522741832157
30.74951953875721
30.954516335682253
31.1595131326073
31.364509929532353
31.569506726457398
31.774503523382446
31.979500320307498
32.18449711723254
32.38949391415759
32.59449071108264
32.79948750800769
33.00448430493274
33.20948110185778
33.41447789878283
33.61947469570788
33.82447149263293
34.029468289557975
34.23446508648303
34.43946188340807
34.644458680333116
34.84945547725817
35.05445227418322
35.259449071108264
35.46444586803331
35.66944266495836
35.874439461883405
36.07943625880846
36.28443305573351
36.48942985265855
36.6944266495836
36.89942344650865
37.104420243433694
37.309417040358746
37.5144138372838
37.71941063420884
37.924407431133886
38.12940422805894
38.33440102498398
38.539397821909034
38.744394618834086
38.94939141575912
39.154388212684175
39.35938500960923
39.56438180653427
39.76937860345932
39.9743754